# Hidden Markov Model Project

- Build a Hidden Markov Model
- Use Hard EM to do semi-supervised part of speech tagging

In [1]:
import nltk
from nltk.corpus import brown
import numpy as np
import scipy
from collections import defaultdict,Counter
from random import shuffle,seed,choice,random
from sklearn.metrics import adjusted_rand_score
nltk.download('universal_tagset')

[nltk_data] Downloading package universal_tagset to
[nltk_data]     /Users/qichao/nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!


True

### Part 1: HMM Initialization and training

We initialize an HMM model in the function `HMM.__init__()`, which takes two arguments a vocabulary `emissions` and a state set `states`. The `HMM` class contains the members

* `self.emissions`, the list of word types, and
* `self.states`, the list of possible POS tags.
* `self.w2i` and `self.i2w`, dictionaries for converting between word tokens like `"dog"` and index numbers like 134.
* `self.s2i` and `self.i2s`, dictionaries for converting between states like `"NOUN"` and index numbers like 12.

We initialize three member variables:

* `self.init_prob`, an `np.array` of initial state probabilities of shape `1 x sixe_of_state_set`.
* `self.emission_prob`, an `np.array` of emission probabilities of shape `size_of_state_set x size_of_vocabulary`. 
* `self.transition_prob` an `np.array` of transition probabilities of shape `size_of_state_set x size_of_state_set`.

All array values are initialized to 0.



We fully supervised train the HMM model in the function `HMM.train()`. The function takes a training set `data`, which is a list of tagged sentences, e.g.:
```
[[("the", "DET"),("dog", "NOUN"),("barks","VERB")], [("the","DET"),("dog","NOUN")]]
```

We then convert words and POS tags in `data` into index numbers using the function `HMM.data2i()`. The output will look something like this: 

```
[[(101, 10), (1000, 5), (5, 2)], [(101, 10), (1000, 5)]]
```

The left element of each pair is an index number corresponding to a word type in the vocabulary and the right element is an index number of a state (i.e. POS tag in our case).

We start the actual training by storing **counts** of emissions, transitions and initial states in `self.emission_prob`, `self.transition_prob` and `self.initial_prob` (these are transition probabilities $P(s|{\rm START})$), respectively. For example, if the word number `101` is emitted twice in the state `10`, then we want `self.emission_prob[10][101] == 2`. Similarly, if we transition from state `10` to state `5` twice in `data`, we want `self.transition_prob[10][5] == 2`.

Then **apply add-one smoothing** to all counts, and normalize probabilities according to:

$$\large P_{initial}(s) = \frac{{\rm count}_{initial}(s)}{\sum_{t} {\rm count}_{initial}(t)}$$

$$\large P_{emission}(w | s) = \frac{{\rm count}(w,s)}{{\rm count}(s)}$$

$$\large P_{transition}(t|s) = \frac{{\rm count}(s,t)}{{\rm count}(s)}$$

Finally, convert all probabilities to log-probabilities using $p \mapsto \log_2 p$ (note that the base of the logarithm is 2). 

In [2]:
from copy import deepcopy

# Symbol used to replace unknown tokens in the input 
UNK="<UNK>"

class HMM:
    def __init__(self, emissions, states):
            # Vocabulary and tag set.
            self.emissions = deepcopy(emissions + [UNK])
            self.states = deepcopy(states)

            # Use these to convert between strings and ID numbers
            self.w2i = {w:i for i, w in enumerate(self.emissions)}
            self.i2w = self.emissions
            self.s2i = {s:i for i,s in enumerate(self.states)}
            self.i2s = self.states

            size_of_state_set = len(self.states)
            size_of_vocabulary = len(self.emissions)

            self.init_prob = np.zeros((1, size_of_state_set))
            self.emission_prob = np.zeros((size_of_state_set, size_of_vocabulary))
            self.transition_prob = np.zeros((size_of_state_set, size_of_state_set))

    def data2i(self, data):
        """ Encode emissions and states into index numbers.
            ex is either a sequence of words or a sequence
            of word-state pairs.
        """
        idx_data = []
        if type(data[0][0]) == type(""):
            for ex in data:
                idx_data.append([])
                for w in ex:
                    w = w if w in self.w2i else UNK
                    idx_data[-1].append(self.w2i[w])
        else:
            for ex in data:
                idx_data.append([])
                for w,s in ex:
                    w = w if w in self.w2i else UNK
                    idx_data[-1].append((self.w2i[w], self.s2i[s]))
        return idx_data

    def train(self, data):
        self.init_prob[:] = 0
        self.emission_prob[:] = 0
        self.transition_prob[:] = 0

        for sentence in self.data2i(data):
            flag = -1
            for word, state in sentence:
                self.emission_prob[state][word] += 1
                if(flag != -1):
                    self.transition_prob[flag][state] +=1
                    flag = state
                if(flag == -1):
                    self.init_prob[0][state] += 1
                    flag = state

        self.init_prob = self.init_prob + 1   
        self.init_prob = np.log2((self.init_prob)/(np.sum(self.init_prob)))
        self.emission_prob= self.emission_prob + 1
        self.transition_prob= self.transition_prob + 1
        for i in range(0,len(self.emission_prob)):
            self.emission_prob[i]= np.log2((self.emission_prob[i])/(np.sum(self.emission_prob[i])))
        for i in range(0,len(self.transition_prob)):
            self.transition_prob[i]= np.log2((self.transition_prob[i])/(np.sum(self.transition_prob[i])))


    def greedy_decode(self, ex):
        """ Greedy (or beam 1 decoding). """
        ex = self.data2i([ex])[0]
        state_distr = np.array(self.init_prob)
        output = []
        log_prob = 0
        for w in ex:
            state_distr += self.emission_prob[:, w].reshape(1,-1)
            output.append(state_distr.argmax())
            log_prob = state_distr.max()
            state_distr = log_prob + self.transition_prob[[output[-1]], :]
        return [(self.i2w[w], self.i2s[s]) for w, s in zip(ex, output)], log_prob

    def extract_output(self, trellis, back_pointers, ex):
        log_prob = trellis[:,-1].max()
        output = [trellis[:,-1].argmax()]
        while len(output) < len(ex):
            output.append(back_pointers[output[-1], len(ex) - len(output)])
        output = output[::-1]

        return [(self.i2w[w], self.i2s[s]) for w, s in zip(ex, output)], log_prob

    def viterbi_decode(self, ex):
        """ Viterbi decoding using loops. """
        ex = self.data2i([ex])[0]
        trellis = np.full((len(self.states), len(ex)), (-float("inf")))
        back_pointers = np.full((len(self.states), len(ex)), -1)
        trellis[:,0] = self.init_prob[0,:] + self.emission_prob[:, ex[0]]

        for i in range(1,len(ex)):
            for j in range(len(self.states)):
                x = []
                for k in range(len(self.states)):
                    x.append(trellis[k,i-1] + self.transition_prob[k,j] + self.emission_prob[j,ex[i]])
                x = np.array(x)
                trellis[j,i] = np.max(x)
                back_pointers[j,i] = np.argmax(x)
        return self.extract_output(trellis, back_pointers, ex)

    def fast_viterbi_decode(self, ex):
        """ Vectorized Viterbi decoding. """
        ex = self.data2i([ex])[0]
        trellis = np.full((len(self.states), len(ex)), (-float("inf")))
        back_pointers = np.full((len(self.states), len(ex)), -1)
        trellis[:,0] = self.init_prob[0,:] + self.emission_prob[:, ex[0]]
        prob = np.full((len(set(self.states)), len(set(self.states))), -float("inf"))
        
        for i in range(1, len(ex)):
            prob = (trellis[:, i-1] + self.transition_prob.T + self.emission_prob[:, ex[i]].reshape(-1, 1))
            trellis[:, i] = np.max(prob, axis = 1)
            back_pointers[:, i] = np.argmax(prob, axis = 1)
        return (self.extract_output(trellis, back_pointers, ex))

### Part 2: Viterbi decoding

In this part, we implement Viterbi decoding in two ways. 


We implement the Viterbi algorithm using loops (i.e. as a non-vectorized algorithm) in the function `HMM.viterbi_decode()`.

The function takes a single argument `ex`, a list representing a sentence, e.g.:
```
["The", "dog", "sleeps"]
```

We start by converting `ex` into a list of index numbers using the function `HMM.data2i()`. Note that the function takes a list of sentences as input instead of a single sentence. 

We then initialize two `np.array` objects: 

* `trellis`, which contains the Viterbi probabilities $v_i(s)$ for each state $s$ and position $i$ in the sentence, and 
* `back_pointers`, which contains back pointers. These identify the optimal tag history.

Both of these need to have dimension `len(self.states) x len(ex)` and we initialize all values in `trellis` to negative infinity (`-float("inf")`) and all values in `back_pointers` to `-1`.

We then start filling the trellis one row at a time:

1. We'll first initialize all elements `trellis[s,0]` to the sum of the initial log-probability of state `s` and the emission log-probability of the first input word `ex[0]` in the given state.
2. When filling in the cell for state `s` in position `i+1`, we need to loop over all states for position `i` and find the state $r_{max}$ which maximizes $\log_2 v_{i}(r) + \log_2 P_{transition}(r,s) + \log_2 P_{emission}(w,s)$, where $w$ is the $i+1$th token in `ex`. This is the Viterbi log-probability $v_{i+i}(s)$.
3. We also store $r_{max}$ in cell `s,i+1` in `back_pointers`.

After we've filled in `trellis` and `back_pointers`, we call the function `self.extract_output()` which will extract the output tag sequence.




We will now implement a vectorized version of Viterbi in the function `HMM.fast_viterbi_decode()`, which again takes a single argument: `ex` representing a sentence. A successfully implemented vectorized Viterbit can be substantially faster than a loop-based approach.

We start by converting word tokens in `ex` into index numbers and intialize `trellis` and `back_pointers` as in Assignment 2.1.

We then initialize the first row of the trellis to the sum of your initial probability vector and emission probabilities for `ex[0]`.

When filling in column `i+1`, we start by computing a `len(self.states) x len(self.states)` matrix `log_probs`, where the element `log_probs[r,s]` represents the log-probability: 

$\log_2 v_{i}(r) + \log_2 P_{transition}(r,s) + \log_2 P_{emission}(w,s)$

where $w$ is the token `ex[i+1]`. 

After we've computed `log_probs`, we need to find the maximal element in each row and assign it to row `i+1` in `trellis`. These will be our Viterbi log-probabilities `v_{i+1}(s)` in row `i+1`. We also need to store the index of the element in `back_pointers`. 



### Part 3: Hard EM

In this part we use hard EM to train an HMM in a semi-supervised manner. We'll use the Brown corpus and form a small manually annotated training set which is combined with a large amount of unlabeled data.

The `accuracy` function below computes tagging accuracy:

In [13]:
def accuracy(sys_data, gold_data):
    """ Compute tagging accuracy. """
    sys_tags = [t for ex in sys_data for w,t in ex]
    gold_tags = [t for ex in gold_data for w,t in ex]
    return 100 * (np.array(sys_tags) == np.array(gold_tags)).sum()/len(gold_tags)

We start by reading the tagged sentences in the Brown corpus using the tag set `"universal"`. We then divide the corpus into a train split `train_set`, containing 80% of the sentences in the corpus, and a test split `test_set` containing the remaining 20%.

To avoid over-representation of any domain in the training and test set, we assign sentences into the train and test splits evenly over the entire corpus. For every consecutive 10 sentences in the Brown corpus, assign 8 to `train_set` and the remaining 2 to `test_set`. E.g. if the sentences in the Brown corpus are $s_1 ... s_n$, then the test set will contain sentences $s_9, s_{10}, s_{19}, s_{20}, s_{29}, s_{30}, ...$ and all remaining sentences will end up in `train_set`. 

We also generate `train_input` and `test_input` which contain untagged versions of the sentences in `train_set` and `test_set`, i.e. simply lists of word tokens. 

In [67]:
from nltk.corpus import brown
tagged_sents = brown.tagged_sents(tagset='universal')
test_set = [[(w.lower(),t) for (w,t) in sent] for i, sent in enumerate(tagged_sents) if i%10==8 or i%10==9]
train_set = [[(w.lower(),t) for (w,t) in sent] for i, sent in enumerate(tagged_sents) if i%10!=8 and i%10!=9]

In [71]:
brown.sents()
test_input = [[w.lower() for w in sent] for i, sent in enumerate(brown.sents()) if i%10==8 or i%10==9]
train_input = [[w.lower() for w in sent] for i, sent in enumerate(brown.sents()) if i%10!=8 and i%10!=9]

We will now generate a small labeled training set `mini_train_set`. Sample every 5000th sentence from `train_set` into the small labeled training set.

We also generate `vocab`, a list of word types occurring in `train_set` and `test_set`, as well as `tags`, a list of unique tags occurring in `mini_train_set`.

In [73]:
# Rate at which we sample examples from our large 
# training set into our small annotated training set
INV_TRAIN_SAMPLING_RATE=5000

mini_train_set = [sent for i, sent in enumerate(train_set) if i%INV_TRAIN_SAMPLING_RATE==0]
print(len(mini_train_set))

vocab = list(set([w for sent in (train_set+test_set) for w,t in sent ]))
print(len(vocab))

tags = list(set([t for sent in mini_train_set for w,t in sent]))
print(len(tags))

10
49815
11


We nitialize an `HMM` object `hmm` using the vocabulary and tagset. Train the model on `mini_train_set`.

Apply inference to the sentences in `test_input` and print tagging accuracy. Initially, we use `HMM.greedy_decode` . Switch to `HMM.fast_viterbi_decode` when it is ready (you should get accuracy close to 50%).

This is out baseline accuracy before we run hard EM.

In [78]:
# your code here
hmm = HMM(vocab, tags)
hmm.train(mini_train_set)
preds_greedy = []
preds_viterbi = []
for sent in test_input:
    output, _ = hmm.greedy_decode(sent)
    output_viterbi, _ = hmm.fast_viterbi_decode(sent)
    preds_greedy.append(output)
    preds_viterbi.append(output_viterbi)
print('The tagging accuracy of HMM.greedy_decode is :', round(accuracy(preds_greedy, test_set)), '%')
print('The tagging accuracy of HMM.fast_viterbi_decode is :', round(accuracy(preds_viterbi, test_set)), '%')

The tagging accuracy of HMM.greedy_decode is : 32 %
The tagging accuracy of HMM.fast_viterbi_decode is : 49 %


When we run hard EM, we will use tag perplexity as stopping criterio. We now implement the function `get_perplexity` which takes a list $\mathcal{D}$ of $N$ tagged sentences $(x_i, y_i)$ and log-probabilities $\log_2 P(x_i, y_i)$ as input. It then returns average per-token tag perplexity as defined by the following formula:

$$\large {\rm PP}(\mathcal{D}) = 2^{- \frac{1}{N} \cdot \sum_{i=1}^N \frac{\log_2 P(x_i,y_i)}{|x_i|}}$$

where $|x_i|$ is the length of sentence $x_i$. 

In [85]:
def get_perplexity(data):
    # your code here
    sum_prob = 0
    for sent, log_prob in data:
        sum_prob += log_prob/len(sent)
    return 2**((-1/len(data))*sum_prob)

Finally, we will implement hard EM. Here we will alternate between:

* the **E-step** where you tag both `train_input` and `test_input` using current HMM parameters, and
* the **M-step** where you retrain the HMM using the tagged output from the E-step.

We'll use perplexity as stopping criterion. Generally the M-step will always reduce perplexity. When this reduction `delta` is smaller than a threshold `PERPLEXITY_TH` (0.1), we will stop the EM algorithm. Start by initializing two variables `old_perplexity` and `delta` (i.e. the change in perplexity) to infinity (`float("inf")`). Also reinitialize an `HMM` object `hmm` using `vocab` and `tags`, and train it on `mini_train_set`.

At every step of the algorithm, you should first tag the entire training set `train_input` and the test set `test_input` using your current parameters for `hmm` (use `HMM.fast_viterbi_decode()` if it's available, or `HMM.greedy_decode()`, otherwise). 

We then compute `perplexity` on the tagged output. Using your old and new perplexity value, compute an updated value for `delta`. Your should also print the current `perplexity` and `delta`. 

If `delta` is less than `PERPLEXITY_TH`, you can stop. Otherwise, use the tagger output for `train_input` and `test_input` to retrain `hmm`.

The perplexity should continuously drop  when using Viterbi, i.e. the perplexity should always be positive (note that this is not necessarily true for `HMM.greedy_decode()`).

In [87]:
PERPLEXITY_TH = 0.1

hmm = HMM(vocab, tags)
hmm.train(mini_train_set)

old_perplexity = float("inf")
delta = float("inf")

while delta > PERPLEXITY_TH:
    pred_train = []
    pred_test = []
    for sent in train_input:
        output_train, _ = hmm.viterbi_decode(sent)
        pred_train.append((output_train, _))
    for sent in test_input:
        output_test, _ = hmm.viterbi_decode(sent)
        pred_test.append((output_test, _))
    
    new_perplexity = get_perplexity(pred_test)
    delta = old_perplexity - new_perplexity
    print('Perplexity', new_perplexity, 'Delta', delta)
    
    old_perplexity = new_perplexity
    new_train_set = [sent for sent,num in (pred_train+pred_test)]
    new_tags = list(set([t for sent in new_train_set for w,t in sent]))

    hmm = HMM(vocab, new_tags)
    hmm.train(new_train_set)

Perplexity 93076.10959004151 Delta inf
Perplexity 1134.3628365389332 Delta 91941.74675350258
Perplexity 1084.83507012278 Delta 49.5277664161531
Perplexity 1064.0816053719457 Delta 20.753464750834382
Perplexity 1051.7114318912902 Delta 12.370173480655467
Perplexity 1039.9612276911428 Delta 11.750204200147436
Perplexity 1031.9109166750316 Delta 8.050311016111209
Perplexity 1024.4768390261677 Delta 7.434077648863877
Perplexity 1018.5216840529934 Delta 5.955154973174331
Perplexity 1013.5455550099454 Delta 4.97612904304799
Perplexity 1009.2370892480266 Delta 4.3084657619187965
Perplexity 1005.6608111121643 Delta 3.5762781358622533
Perplexity 1002.7812124225209 Delta 2.8795986896434442
Perplexity 1000.8081805225702 Delta 1.9730318999506835
Perplexity 999.2727822984341 Delta 1.5353982241360882
Perplexity 998.0668605625979 Delta 1.2059217358362275
Perplexity 997.2448098339612 Delta 0.8220507286366683
Perplexity 996.6971237081076 Delta 0.5476861258536019
Perplexity 996.141249934898 Delta 0.5558

Finally, we tag `test_input` using your EM-trained `hmm` and evaluate accuracy.

In [88]:
preds_viterbi = []
for sent in test_input:
    output_viterbi, _ = hmm.viterbi_decode(sent)
    preds_viterbi.append(output_viterbi)
print('The tagging accuracy of EM-trained hmm is :', round(accuracy(preds_viterbi, test_set)), '%')

The tagging accuracy of EM-trained hmm is : 57 %
